# How to painlessly transform an NLP model in Jupyter to a production API?

# [SMS Spam Collection Data Set](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection)

<img src="https://archive.ics.uci.edu/ml/assets/logo.gif" align='left' />

# Bag-of-words model with word embeddings learned from scratch

<a href="https://colab.research.google.com/github/Paulescu/practical-nlp-2021/blob/main/spam_detection/noteboooks/model.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" align="left"/>
</a>

### Required setup if you run the notebook in Google Colab

In [1]:
# # you need to paste the URL of your Github repo here if you want to run this notebook in Google Colab.
# URL_GITHUB_REPO = 'https://github.com/Paulescu/practical-nlp-2021'

# # a hacky way to check if the current notebook is running in Google Colab.
# if 'google.colab' in str(get_ipython()):
#     # we are running notebook in Colab
#     !git clone $URL_GITHUB_REPO
#     !cd .. && python setup.py develop
# else:
#     print('Python setup skiped.')

# Step 1. Download data and split into train, validation and test

The dataset can be found [here](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection)

### Download raw data

In [1]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
!tar -xf smsspamcollection.zip

--2020-12-10 18:51:54--  https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 203415 (199K) [application/x-httpd-php]
Saving to: ‘smsspamcollection.zip.2’

smsspamcollection.z 100%[===================>] 198.65K   271KB/s    in 0.7s    

2020-12-10 18:51:56 (271 KB/s) - ‘smsspamcollection.zip.2’ saved [203415/203415]



### Quick data exploration

In [2]:
import pandas as pd

data = pd.read_csv('SMSSpamCollection', sep='\t', header=None)
data.columns = ['label', 'text']

In [3]:
data.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
data['label'].value_counts(normalize=True)

ham     0.865937
spam    0.134063
Name: label, dtype: float64

### Add numeric column for the label

In [5]:
IDX_TO_LABEL = {
    0: 'ham',
    1: 'spam',
}

LABEL_TO_IDX = {
    'ham': 0,
    'spam': 1,
}

data['label_int'] = data['label'].apply(lambda x: LABEL_TO_IDX[x])
data.head()

,label,text,label_int
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


### Split data into files `train.csv` , `validation.csv`, `test.csv`

In [6]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, test_size=0.20, random_state=123,)
train_data, validation_data = train_test_split(train_data, test_size=0.20, random_state=123)

print('train_data: ', len(train_data))
print('validation_data: ', len(validation_data))
print('test_data: ', len(test_data))

train_data[['label_int', 'text']].to_csv('train.csv', index=False, header=False)
validation_data[['label_int', 'text']].to_csv('validation.csv', index=False, header=False)
test_data[['label_int', 'text']].to_csv('test.csv', index=False, header=False)

train_data:  3565
validation_data:  892
test_data:  1115


# Step 2. Define PyTorch `DataLoader`s for train, validation, and test.

In [7]:
import spacy
try:
    spacy_eng = spacy.load('en')
except:
    print('Downloading spacy resources for English')
    !python -m spacy download en
    spacy_eng = spacy.load('en')

In [8]:
import torch
from torchtext.data import Field, TabularDataset, BucketIterator

def tokenizer_fn(text):
    # Input: "Come to see me!"
    # Output: "['Come', 'to', 'see', 'me', '!']"
    return [tok.text for tok in spacy_eng.tokenizer(text)]

# Create PyTorch Datasets from train.csv, validation.csv, test.csv
TEXT = Field(sequential=True, tokenize=tokenizer_fn, lower=True, batch_first=True)
LABEL = Field(sequential=False, use_vocab=False)

train, validation, test = TabularDataset.splits(
    path='',
    train='train.csv',
    validation='validation.csv',
    test='test.csv',
    format='csv',
    skip_header=False,
    fields=[('label', LABEL), ('text', TEXT)],
)

# Build the vocabulary using the train dataset
TEXT.build_vocab(train, max_size=10000, min_freq=2)
vocab_size = len(TEXT.vocab) # we will need this later
print('Vocabulary size: ', vocab_size)

# Create PyTorch DataLoaders for train, validation, test
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 128
train_iter, validation_iter, test_iter = BucketIterator.splits(
    (train, validation, test),
    batch_sizes=(BATCH_SIZE, BATCH_SIZE, BATCH_SIZE),
    device=DEVICE,
    sort_key=lambda x: len(x.text),
    sort_within_batch=True,
)

/Users/paulabartabajo/src/online-courses/practical-nlp-2021/spam_detection/.venv/lib/python3.7/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/Users/paulabartabajo/src/online-courses/practical-nlp-2021/spam_detection/.venv/lib/python3.7/site-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
/Users/paulabartabajo/src/online-courses/practical-nlp-2021/spam_detection/.venv

Vocabulary size:  3337


/Users/paulabartabajo/src/online-courses/practical-nlp-2021/spam_detection/.venv/lib/python3.7/site-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


#### Check output from `Dataloader`

In [9]:
train_input = next(iter(train_iter))

print(train_input.text)
print(train_input.label)

tensor([[  46,    4,   27,  ..., 2665,   16,  331],
        [  85,  131,    2,  ...,  131,  635,   15],
        [  13,   98,  333,  ...,    0,    0,    2],
        ...,
        [  24,  169,  278,  ...,  249,    1,    1],
        [ 393,    0,  279,  ...,   11,    1,    1],
        [ 130,   21,   55,  ...,    2,    1,    1]])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])


/Users/paulabartabajo/src/online-courses/practical-nlp-2021/spam_detection/.venv/lib/python3.7/site-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [10]:
for i in range(10):
    print('text: ', train[i].text)
    print('label: ', train[i].label)
    print('---')

text:  ['mom', 'wants', 'to', 'know', 'where', 'you', 'at']
label:  0
---
text:  ['boy', ';', 'i', 'love', 'u', 'grl', ':', 'hogolo', 'boy', ':', 'gold', 'chain', 'kodstini', 'grl', ':', 'agalla', 'boy', ':', 'necklace', 'madstini', 'grl', ':', 'agalla', 'boy', ':', 'hogli', '1', 'mutai', 'eerulli', 'kodthini', '!', 'grl', ':', 'i', 'love', 'u', 'kano;-', ')']
label:  0
---
text:  ['its', 'on', 'in', 'engalnd', '!', 'but', 'telly', 'has', 'decided', 'it', 'wo', "n't", 'let', 'me', 'watch', 'it', 'and', 'mia', 'and', 'elliot', 'were', 'kissing', '!', 'damn', 'it', '!']
label:  0
---
text:  ['your', 'gon', 'na', 'have', 'to', 'pick', 'up', 'a', '$', '1', 'burger', 'for', 'yourself', 'on', 'your', 'way', 'home', '.', 'i', 'ca', "n't", 'even', 'move', '.', 'pain', 'is', 'killing', 'me', '.']
label:  0
---
text:  ['no', 'no:)this', 'is', 'kallis', 'home', 'ground.amla', 'home', 'town', 'is', 'durban', ':', ')']
label:  0
---
text:  ['i', 'am', 'seeking', 'a', 'lady', 'in', 'the', 'street', 

# Step 3. Define the neural net model

In [11]:
# TODO: add diagram here

In [16]:
import torch.nn as nn
import torch.nn.functional as F
    
class Model(nn.Module):
    
    def __init__(self, vocab_size: int, embedding_dim: int):
        super(Model, self).__init__()
        self.embed = nn.Embedding(vocab_size, embedding_dim)
        self.global_avg_pooling = lambda x: torch.mean(x, dim=-2)
        self.fc1 = nn.Linear(embedding_dim, 16)
        self.fc2 = nn.Linear(16, 2)
        
    def forward(self, x):
        x = self.embed(x)
        x = self.global_avg_pooling(x)

        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x

EMBEDDING_DIM = 16
model = Model(vocab_size, EMBEDDING_DIM).to(DEVICE)

# Step 4. Train the model

### Loss function and optimizer

In [17]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-4)

### Launch Tensorboard

In [18]:
%load_ext tensorboard
%tensorboard --logdir runs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6008 (pid 79472), started 1:26:00 ago. (Use '!kill 79472' to kill it.)

### Train loop

In [19]:
# Setup logging to Tensorboard
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

now = datetime.now()
now = now.strftime("%Y-%m-%d-%H:%M:%S")
MODEL_NAME = 'bag_of_words_embeddings_from_scratch'
log_file = f'./runs/{MODEL_NAME}/{now}'
writer = SummaryWriter(log_file)

# Train lopp
from tqdm import tqdm
N_EPOCHS = 150
for epoch in range(N_EPOCHS):
    
    # train
    running_loss = 0.0
    model.train()
    train_size = 0
    running_accuracy = 0.0
    for batch in tqdm(train_iter):
        
        # forward pass to compute the batch loss
        x = batch.text
        y = batch.label.long()
        predictions = model(x)        
        loss = criterion(predictions, y)
            
        # backward pass to update model parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # compute train metrics
        running_loss += loss.data * x.size(0)
        _, predicted_classes = torch.max(predictions, 1)
        running_accuracy += predicted_classes.eq(y.data).sum().item()
        train_size += x.size(0)
        
    epoch_loss = running_loss / train_size
    epoch_accuracy = running_accuracy / train_size
    
    # validation
    val_loss = 0.0
    model.eval()
    val_size = 0
    val_accuracy = 0
    with torch.no_grad():
        for batch in validation_iter:
            x = batch.text
            y = batch.label.long()
            predictions = model(x)
            loss = criterion(predictions, y)
            
            # compute validation metrics
            val_loss += loss.data * x.size(0)
            _, predicted_classes = torch.max(predictions, 1)
            val_accuracy += predicted_classes.eq(y.data).sum().item()           
            val_size += x.size(0)
            
        val_loss /= val_size
        val_accuracy /= val_size
        
        print('\nEpoch: {}'.format(epoch))
        print('Loss \t Train: {:.4f} \t Validation: {:.4f}'.format(epoch_loss, val_loss))
        print('Acc: \t Train: {:.4f} \t Validation: {:.4f}'.format(epoch_accuracy, val_accuracy))

    # log metrics to tensorboard
    writer.add_scalars('Loss', {'train': epoch_loss, 'validation': val_loss}, epoch + 1)
    writer.add_scalars('Accuracy', {'train': epoch_accuracy, 'validation': val_accuracy}, epoch + 1)
    
writer.close()

 43%|████▎     | 12/28 [00:00<00:00, 112.39it/s]


Epoch: 0
Loss 	 Train: 0.7787 	 Validation: 0.7616
Acc: 	 Train: 0.1380 	 Validation: 0.1155


 25%|██▌       | 7/28 [00:00<00:00, 65.49it/s]


Epoch: 1
Loss 	 Train: 0.7566 	 Validation: 0.7395
Acc: 	 Train: 0.1386 	 Validation: 0.1244


 46%|████▋     | 13/28 [00:00<00:00, 125.95it/s]

...


Epoch: 147
Loss 	 Train: 0.0253 	 Validation: 0.0474
Acc: 	 Train: 0.9927 	 Validation: 0.9832


 43%|████▎     | 12/28 [00:00<00:00, 111.02it/s]


Epoch: 148
Loss 	 Train: 0.0250 	 Validation: 0.0471
Acc: 	 Train: 0.9927 	 Validation: 0.9832


100%|██████████| 28/28 [00:00<00:00, 109.83it/s]


Epoch: 149
Loss 	 Train: 0.0244 	 Validation: 0.0469
Acc: 	 Train: 0.9933 	 Validation: 0.9832


# Step 5. Test the model

In [20]:
test_accuracy = 0.0
test_size = 0
with torch.no_grad():
    for batch in test_iter:
        # forward pass
        x = batch.text
        y = batch.label.long()
        predictions = model(x)        
        loss = criterion(predictions, y)

        # compute accuracy
        _, predicted_classes = torch.max(predictions, 1)
        test_accuracy += predicted_classes.eq(y.data).sum().item()
        test_size += x.size(0)

test_accuracy /= test_size
print('Test accuracy: {:.4f}'.format(test_accuracy))

Test accuracy: 0.9731


# Extra. Interact with the model
Pay attention how pre-processing and post-processing are necessary to be able to use the model at inference time.

https://github.com/bentrevett/pytorch-sentiment-analysis/issues/40

In [21]:
sentences = [
    'This is your friend Carl. Come to the Casino and get a discount!',
    'This is your friend Carl, do you want to meet later?',
    'Would you be interested in buying a car for nothing?',
    'Send your card details today and get a prize!',
    'I won two tickets to the show, do you want to come with me?',
    'I won two tickets to the show, just send an SMS to this number and get them',
]

for s in sentences:
    # pre-process text into integer tokens
    model_input = [TEXT.vocab.stoi[token] for token in tokenizer_fn(s)]
    # add 0-dimension
    model_input = torch.LongTensor(model_input).unsqueeze(0)
    
    # run model prediction
    predictions = model(model_input)
    
    # post-processing
    _, predicted_class = torch.max(predictions, 1)
    predicted_class = predicted_class.item()
    predicted_class_str = IDX_TO_LABEL[predicted_class]
    
    # print
    print(s)
    print(predicted_class_str)
    print('------')

This is your friend Carl. Come to the Casino and get a discount!
ham
------
This is your friend Carl, do you want to meet later?
ham
------
Would you be interested in buying a car for nothing?
ham
------
Send your card details today and get a prize!
spam
------
I won two tickets to the show, do you want to come with me?
ham
------
I won two tickets to the show, just send an SMS to this number and get them
ham
------


# Extra: Visualize the learned word embeddings with the [Embedding Projector](https://projector.tensorflow.org/)

### Extract embedding parameters

In [ ]:
for name, parameter in model.named_parameters():
    if name == 'embed.weight':
        embeddings = parameter

print(embeddings.shape)

### Generate tsv files

In [ ]:
import io

embeddings = embeddings.cpu().detach().numpy()
vocab = TEXT.vocab.itos

out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
    if index in [0, 1]:
        # skip 0, it's the unknown token.
        # skip 1, it's the padding token.
        continue
        
    vec = embeddings[index, :] 
    out_v.write('\t'.join([str(x) for x in vec]) + "\n")
    out_m.write(word + "\n")

out_v.close()
out_m.close()

### Download files to your local computer (in case you are running this notebook in Google Colab)

In [ ]:
try:
    from google.colab import files
    files.download('vectors.tsv')
    files.download('metadata.tsv')
except Exception as e:
    pass